# Web Scraping e Integração com BigQuery
Este notebook visa apenas detalhar o código ```steam_db_sales_scraper.py``` de entrega continua do web scraping no site SteamDB para coletar dados de vendas de jogos no Steam e envia esses dados para o Google BigQuery.

O processo é dividido em várias etapas:
- **Atualização de Cookies**: Coleta de cookies necessários para burlar o Cloudflare.
- **Coleta de Dados**: Scrape das informações de vendas de jogos no Steam.
- **Armazenamento no BigQuery**: Envio dos dados coletados para o Google BigQuery.

Vamos começar com a instalação das bibliotecas necessárias e a configuração do ambiente.

In [ ]:
# Instalar bibliotecas necessárias (se ainda não estiverem instaladas)
pip install -r requirements.txt

### 1. Carregar as variáveis de ambiente
Primeiro, carregamos as variáveis de ambiente usando o arquivo `.env` que contém as chaves de acesso à API e credenciais do Google Cloud.

O uso do scrapedo foi apenas para ter proxies rotativos para cada requisição evitando o bloqueio por IP do CloudFlare, poderiamos usar outras ferramentas de rotatividade de ip ou proxie.

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

GOOGLE_APPLICATION_CREDENTIALS = os.getenv('GOOGLE_APPLICATION_CREDENTIALS')
PROJECT_ID = os.getenv('PROJECT_ID')
DATASET_ID = os.getenv('DATASET_ID')
TABLE_ID = os.getenv('TABLE_ID')
SCRAPEDO_API_KEY = os.getenv('SCRAPEDO_API_KEY')

print(f'GOOGLE_APPLICATION_CREDENTIALS: {GOOGLE_APPLICATION_CREDENTIALS}')
print(f'PROJECT_ID: {PROJECT_ID}')

### 2. Função para Atualizar os Cookies
A função `atualizar_cookies` é responsável por obter os cookies necessários para contornar a proteção do Cloudflare e permitir o acesso ao site SteamDB. Ela utiliza o módulo `CloudflareBypasser` para contornar a verificação de segurança.

O site, por ser protegido por CloudFlare, as requisicoes sao validadas a partir de 2 cookies `cf_bm` e `cf_Clearence`, então eu forço o cloudflare no site da steam ao acessar `/cloudflare`, uso o bypass e consigo os cookies para validar as requisições.

Seria mais interessante tornar o ByPass como um serviço para ser chamado e retornar o cookies

In [ ]:
import time
import random
from CloudflareBypasser import CloudflareBypasser
from DrissionPage import ChromiumPage, ChromiumOptions
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def atualizar_cookies(url="https://steamdb.info/cloudflare"):
    logging.info("Iniciando atualização de cookies.")
    tentativas = 0
    max_tentativas = 5
    cookies_necessarios = {}

    while tentativas < max_tentativas:
        logging.info(f"Tentativa {tentativas + 1} de {max_tentativas}.")
        options = ChromiumOptions()
        options.incognito(True)
        driver = ChromiumPage(options)
        try:
            driver.get(url)
            time.sleep(random.randint(5, 10))
            cf_bypasser = CloudflareBypasser(driver)
            cf_bypasser.click_verification_button()
            time.sleep(3)

            cookies = driver.cookies()
            logging.info(f"Cookies coletados: {cookies}")
            for cookie in cookies:
                if cookie['name'] == 'cf_clearance':
                    cookies_necessarios['cf_clearance'] = cookie['value']
                elif cookie['name'] == '__cf_bm':
                    cookies_necessarios['__cf_bm'] = cookie['value']

            if 'cf_clearance' in cookies_necessarios and '__cf_bm' in cookies_necessarios:
                return cookies_necessarios

        except Exception as e:
            logging.error(f"Erro durante a tentativa de obter cookies: {e}")

        finally:
            driver.close()

        tentativas += 1
        logging.warning(f"Tentativa {tentativas}: Cookie cf_clearance não encontrado, tentando novamente...")

    logging.error("Não foi possível obter o cookie cf_clearance após 5 tentativas.")
    raise Exception("Não foi possível obter o cookie cf_clearance após 5 tentativas.")

### 3. Coleta de Dados de Vendas do Steam
A função `obter_dados_steam_sales` realiza o scraping dos dados de vendas do SteamDB. Ela acessa o site SteamDB, coleta as informações de cada jogo, como nome, desconto, preço, classificação, entre outros, e organiza esses dados em um `DataFrame` do Pandas.

Eu implementei uma função a mais que pegaria mais dados detalhados para cada jogo, mas é feita uma requisição por jogo, isso eu fiquei limitado pelo scrapedo, e eu precisaria de uma lista de proxies para gerar uma maior rotatividade de ips



In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import logging
import re

def obter_dados_steam_sales():
    url = "https://steamdb.info/sales/"
    headers = {
        "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
        "accept-language": "pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7",
        "cache-control": "max-age=0",
        "priority": "u=0, i",
        "sec-ch-ua": "\"Google Chrome\";v=\"129\", \"Not=A?Brand\";v=\"8\", \"Chromium\";v=\"129\"",
        "sec-ch-ua-arch": "\"x86\"",
        "sec-ch-ua-bitness": "\"64\"",
        "sec-ch-ua-full-version": "\"129.0.6668.100\"",
        "sec-ch-ua-full-version-list": "\"Google Chrome\";v=\"129.0.6668.100\", \"Not=A?Brand\";v=\"8.0.0.0\", \"Chromium\";v=\"129.0.6668.100\"",
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-model": "\"\"",
        "sec-ch-ua-platform": "\"Linux\"",
        "sec-ch-ua-platform-version": "\"6.8.0\"",
        "sec-fetch-dest": "document",
        "sec-fetch-mode": "navigate",
        "sec-fetch-site": "none",
        "sec-fetch-user": "?1",
        "upgrade-insecure-requests": "1"
    }

    # Realizando a requisição GET para obter os dados da página
    response = requests.get(url, headers=headers)
    response.raise_for_status()

    # Criando um objeto BeautifulSoup para fazer o parse do HTML
    soup = BeautifulSoup(response.content, "html.parser")

    # Buscando a tabela de jogos
    jogos = []
    tabela_jogos = soup.find("table", class_="table-bordered")
    linhas = tabela_jogos.find_all("tr")[1:]  # Pulando o cabeçalho da tabela

    for linha in linhas:
        colunas = linha.find_all("td")
        if len(colunas) > 6:
            nome_jogo = colunas[1].get_text(strip=True)
            desconto = colunas[2].get_text(strip=True)
            preco_atual = colunas[3].get_text(strip=True)
            preco_original = colunas[4].get_text(strip=True)
            data_fim = colunas[5].get_text(strip=True)
            link = colunas[1].find('a')['href']
            jogos.append({
                'nome_jogo': nome_jogo,
                'desconto': desconto,
                'preco_atual': preco_atual,
                'preco_original': preco_original,
                'data_fim': data_fim,
                'link': link,
                'data_coleta': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            })

    # Convertendo para DataFrame
    df = pd.DataFrame(jogos)
    return df

### 4. Envio dos Dados para o Google BigQuery
Agora, vamos criar uma função que envia os dados coletados para o Google BigQuery, utilizando a biblioteca `google-cloud-bigquery`.

In [5]:
from google.cloud import bigquery
import pandas as pd
import logging

def enviar_para_bigquery(df):
    client = bigquery.Client.from_service_account_json(GOOGLE_APPLICATION_CREDENTIALS)
    table_id = f'{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}'
    try:
        # Enviar os dados para o BigQuery
        job = client.load_table_from_dataframe(df, table_id)
        job.result()  # Aguardar o carregamento
        logging.info(f'Dados enviados com sucesso para o BigQuery: {table_id}')
    except Exception as e:
        logging.error(f'Erro ao enviar dados para o BigQuery: {e}')

### 5. Comentários sobre minhas abordagens

- Eu sempre tento deixar o código da forma mais escalável possível, quis evitar ao maximo uso de Selenium ou algo do tipo, por isso priorizei conseguir realizar requests

- Ao perceber o CloudFlare no site, eu já tinha essa ferramenta de ByPass que funciona e consegui capturar os cookies do cloudflare. Como mencionado, eu tinha a ideia de tornar o ByPass como um servico separado, mesmo que ele não consiga ser feito no navegador em modo headless. Pelo timestamp que o cookie tem, eles duram 3 horas.

- Decidi colocar todas as variaveis sensiveis com .env por motivos de segurança

- Implementei diversos logs ao longo do processo de coleta, tratamento e ingestao dos dados, de forma que fique claro onde ocorreu algum erro. Logs esses que podem ser enviados como mensagem para um healthchecks ou slack para manutenção.

- O código está pronto para rodar em um serviço continuo, como por exemplo, colocar num container Docker, ao separar o Bypass como servico, e rodar periodicamente atraves de cronjobs. 

- Fiz um pequeno dashboard em streamlit para conectar o banco de dados da google e implementei algumas analises dos dados.
